NOTE: please view comments regarding .py files as being interchangable with
.ipynb files in these notebooks. Because we are writing to .py files in our
example, but using .ipynb files for illustration here, sometimes we
reference .py files; the developer reading this should recognize those comments
as referring to their respective .ipynb files in this notebooks directory.


NOTE: THE FOLLOWING, FINAL CODE IS EXECUTED IN THE FILE "param_checker_unit_tests.py".

Next, we invoke contract methods by first instantiating an instance of our new contract object, then calling its methods.
We are doing this here as unit tests, which will illustrate a few features of our SDK. We do this in a new .py file titled "param_checker_unit_tests.py".

NOTE: even though here we are using a .ipynb file for illustrative purposes, we
would actually be importing from a file titled "generated_simba_hinted_contract.py"

We need to import our new smart contract python class, which in this case was called "Application."
Note also that we would be importing from "generated_simba_hinted_contract.py"

In [1]:
import unittest
from generated_simba_hinted_contract import Application

ModuleNotFoundError: No module named 'generated_simba_hinted_contract'

Now we will run our unit tests, which illustrate how different method calls are made for our contract

In [ ]:
class TestSimbaOutput(unittest.TestCase):

    a = Application()

    def test_view_param_restrictions(self):
        # comment out this function to prevent printing of parameter restrictions
        print('parameter restrictions:')
        print(TestSimbaOutput.a.simba_contract.params_restricted)

    def test_fail_negative_array_input(self):
        # following should fail because elements must be uints
        arr1 = [-10,4,6,8]
        arr2 = [1,3,5,10]
        with self.assertRaises(ValueError):
            TestSimbaOutput.a.two_arrs(arr1, arr2)

    def test_fail_non_uint_input(self):
        # following should fail because this method's array params should only accept
        arr1 = [2, 4, 'hello', 10]
        arr2 = [1,3,5]
        with self.assertRaises(TypeError):
            TestSimbaOutput.a.two_arrs(arr1, arr2)

    def test_pass_unequal_lengths(self):
        # should pass - two_arrs does not have length requirements on arrays
        arr1 = [2, 4, 20, 10, 3, 3]
        arr2 = [1,3,5]
        status_code = TestSimbaOutput.a.two_arrs(arr1, arr2).status_code
        self.assertEqual(status_code, 202)

    def test_fail_incorrect_outer_array_lengths(self):
        # should fail - nested_arr_3 must contain 3 arrays
        arr1 = [[1,2,3], [2,2,2]]
        with self.assertRaises(ValueError):
            TestSimbaOutput.a.nested_arr_3(arr1)

    def test_fail_incorrect_inner_array_lengths(self):
        # should fail - nested_arr_3's arrays must contain 3 elements
        arr1 = [[1,2,3], [99,99], [13,13,13]]
        with self.assertRaises(ValueError):
            TestSimbaOutput.a.nested_arr_3(arr1)

    def test_fail_mixed_element_data_types(self):
        # should fail - array element data types cannot be mixed in solidity
        arr1 = [[2,2,2], 'hello', [13,13,13]]
        with self.assertRaises(TypeError):
            TestSimbaOutput.a.nested_arr_3(arr1)

    def test_fail_too_many_dimensions(self):
        # should fail - too many dimensions
        arr1 = [1,2,3]
        arr2 = [[arr1,arr1,arr1], [arr1,arr1,arr1], [arr1,arr1,arr1]]
        with self.assertRaises(ValueError):
            TestSimbaOutput.a.nested_arr_3(arr2)

if __name__ == '__main__':
    unittest.main()

Now if we view our App in SEP, we can see that only one of these methods was invoked (test_pass_unequal_lengths).

The reason only one of those methods was invoked is that we passed invalid parameters for those other method calls, which illustrates a really cool aspect of our SDK: it enforces array length and uint requirements for the contract from which our contract class was derived. More specifically, if a parameter is a fixed-length array, a uint (int > 0), a dynamic (non-fixed-length) array that contains uint types, or a fixed-length array that contains uint types, then the relevant restrictions for those parameters are enforced.

To explain this, we need to understand how restrictions for our contract’s methods are stored. A SimbaHintedContract object has an attribute titled params_restricted. For this particular object / contract, params_restricted looks like:

In [ ]:
{'an_arr':
     {'array_params': {'first': {0: None, 'contains_uint': True}}},
'two_arrs':
    {'array_params': {'first': {0: None, 'contains_uint': True}, 'second': {0: None, 'contains_uint': True}}},
 'nested_arr_0':
     {'array_params': {'first': {0: None, 1: None, 'contains_uint': True}}},
 'nested_arr_1':
     {'array_params': {'first': {0: None, 1: 5, 'contains_uint': True}}},
 'nested_arr_2':
     {'array_params': {'first': {0: 4, 1: None, 'contains_uint': True}}},
 'nested_arr_3':
     {'array_params': {'first': {0: 3, 1: 3, 'contains_uint': True}}},
 'nested_arr_4':
     {'array_params': {'first': {0: 3, 1: 3, 'contains_uint': True}}}
 }

Since we are purposefully passing failing parameters to nested_arr_3, let’s take a look at what’s going on with that method, this params_restricted dictionary, and our method calls. First, if you look at the original metadata, the method nested_arr_3 looks like:

In [ ]:
{"nested_arr_3":
    {"params":
        [{
            "name": "first",
            "type": "uint256[3][3]"
        }]
    }
}

What’s indicated by “type”: “uint256[3][3]” is that our param “first” should be an array of length 3, containing arrays of length 3. The entries in these inner arrays should be uint256 types. In Python, we simply enforce that these values must be of type int >= 0.

As to how our SimbaHintedObject keeps this information in state, if we look inside our params_restricted dictionary, we will see the following:

'nested_arr_3': {'array_params': {'first': {0: 3, 1: 3, 'contains_uint': True}}}

The 0:3 entry for “first” means that our outermost array dimension / layer of our parameter must be of length three. Similarly, the 1:3 entry means that the second dimension/layer of our parameter must be of length 3. Finally, the ‘contains_int’:True key value pair is pretty self-explanatory: the innermost arrays in our parameter must contain uint types.

It is encouraged to go through the method invocations in our unit tests and see why all but one failed. Doing so will make clear that we are enforcing a number of restrictions, such as not allowing mixing of element types within arrays, since Solidity does not allow for mixing of element types within arrays.



